In [1]:
from rake_cn import * #which version

In [2]:
def read_short_text(path='./samples/4.cn_property_burst.txt'):
    f=open(path)
    text=f.readlines()
    text=''.join(text)
    return text

In [3]:
def print_list_text(text_list):
    for item in text_list:
        if(type(item)==tuple):
            print item[0],item[1]
        else:
            print item
def return_segs(line):
    tk=tokenizer()
    tokens=tk.tokenize(line)
    natures=tk.tokenize_nature(line)
    return zip(tokens,natures)

In [4]:
def extract_keyword(text,limit=5):
    sentences=split_sentences(text)
    phrase_list = generate_candidate_keywords(sentences, stop_words_pattern)
    word_scores = calculate_word_scores(phrase_list)
    keyword_candidates = generate_candidate_keyword_scores(phrase_list, word_scores)
    sorted_keywords = sorted(keyword_candidates.iteritems(), key=operator.itemgetter(1), reverse=True)
    sorted_keywords_filtered = filter(lambda x: len(x[0])>3*1 and len(x[0])<3*9 and '\n' not in x[0],sorted_keywords)
    return sorted_keywords_filtered[:limit]
    


In [6]:
text = read_short_text('./samples/4.cn_property_burst.txt')

In [11]:
stop_words_pattern=build_stop_word_regex('./stopwords/cn_stopword_list_1208.txt')

In [ ]:
sentences=split_sentences(text)
phrase_list = generate_candidate_keywords(sentences, stop_words_pattern)
word_scores = calculate_word_scores(phrase_list)
keyword_candidates = generate_candidate_keyword_scores(phrase_list, word_scores)


In [ ]:
print_list_text(phrase_list)

In [10]:
sorted_keywords = sorted(keyword_candidates.iteritems(), key=operator.itemgetter(1), reverse=True)

In [11]:
sorted_keywords_filtered = filter(lambda x: len(x[0])>2*1 and len(x[0])<3*9 and '\n' not in x[0],sorted_keywords)

In [12]:
for item in sorted_keywords_filtered[:18]:
    print item[0],item[1]

减少居民购房套数 13.8
揭秘房价崩溃 12.7142857143
资产价位低 8.4
基础设施投资 8.25
金融调控放宽时 7.5
投资渠道少 7.25
线城市厦门 7.0
调控措施抑制房价 6.46428571429
资产价格泡沫 6.22142857143
产热崩溃 5.0
资产价格 4.65
收紧时 4.5
刺激住房 4.33333333333
投资渠道 4.25
央行放宽 4.0
够健全 4.0
华工作 4.0
抵押贷款利率 4.0


## adding extract rules

In [5]:
import pandas as pd

In [6]:
tk=tokenizer()

In [74]:
out_list=[]
for item in sorted_keywords_filtered[:20]:   
    re = return_segs(item[0])
    natures=pd.DataFrame(re)[1].tolist()
    if(('n' in natures or 'NNT' in natures) and 'v' in natures):
        out_list.append(item)
        if(natures.count('v')>1 and (natures.count('n') + natures.count('NNT'))==1): ## 不出现两个动词
            out_list.remove(item)
        if(natures[-1]=='d'):   ## 不以副词结尾
            out_list.remove(item)
        if(natures[0]=='j'): ## 简称略语 开头
            out_list.remove(item)
print_list_text(out_list)

减少居民购房套数 13.8
调控措施抑制房价 6.46428571429
刺激住房 4.33333333333
相关图集 4.0


In [33]:
def check_rules(sorted_keywords_filtered):
    out_list=[]
    for item in sorted_keywords_filtered[:20]:
        try:
            re = return_segs(item[0])
            natures=pd.DataFrame(re)[1].tolist()
            words=pd.DataFrame(re)[0].tolist()
        except:
            continue
        if(('n' in natures or 'NNT' in natures) and 'v' in natures):
            out_list.append(item)
            if(natures.count('v')>1 and (natures.count('n') + natures.count('NNT'))==1): ## 不出现两个动词
                out_list.remove(item)
                continue
            if(natures[-1]=='d'):  ## 不以副词结尾
                out_list.remove(item)
                continue
            if(natures[0]=='j'): ## 简称略语 开头
                out_list.remove(item)
                continue
            if(len(words[0])==1 and natures[0]!='a'):# 第一个字,不可以不是形容词
                out_list.remove(item)
                continue
            if(len(words[-1])==1 and natures[-1]=='v'):#最后一个字,不可以是动词
                out_list.remove(item)
                continue
            if(len(words[-1])==1 and natures[-1]=='a'):#最后一个字,不可以是形容词
                out_list.remove(item)
                continue
    print_list_text(out_list)
    return out_list
noun_set=set(['nw','ns','n','vn','NNT','FINANCIAL','STOCK'])
def check_rules2(sorted_keywords_filtered):
    tk = tokenizer()
    out_list=[]
    for item in sorted_keywords_filtered[:20]:
        try:
            re = return_segs(item[0])
            re = pd.DataFrame(re)
            re.columns=['w','n']
        except:
            continue
        out=None
        for row in re.itertuples():
            nat=row.n
            word=row.w
            if(nat in noun_set):
                if(len(word)==1):
                    continue
                if(out is not None):
                    out=out+word
                else:
                    out=word
            if(nat not in noun_set):
                if(out is not None):
                    out_list.append(out)
                    out=None
                out=None
        if(out is not None):
            out_list.append(out)
    for item in out_list:
        #print item,len(tk.tokenize(item))
        if(len(item)==3 and len(tk.tokenize(item))>1):
            out_list.remove(item)
    #print_list_text(out_list)
    return out_list

def check_rules3(sorted_keywords_filtered):  ## 包含名词, 不以单个词开头或结尾,不包含完整主谓宾
    tk = tokenizer()
    out_list=[]
    for item in sorted_keywords_filtered[:20]:
        try:
            if(item[1]<4): ##score threshold
                continue
            re = return_segs(item[0])
            re = pd.DataFrame(re)
            re.columns=['w','n']
        except:
            continue
        out=None
        none_flag = False
        words=[]
        nats=[]
        for row in re.itertuples():
            nat=row.n
            word=row.w
            words.append(word)
            nats.append(nat)
            if(nat in noun_set):
                none_flag = True
        if(nats[-1]=='an' or nats[-1]=='a'):
            words=words[:-1]
            nats=nats[:-1]
            if(len(words)<1):
                continue
        if(len(words[-1])==1):
            words=words[:-1]
            nats=nats[:-1]
            if(len(words)<1):
                continue
        if(len(words[0])==1):
            words=words[1:]
            nats=nats[1:]
            if(len(words)<1):
                continue
        elif(nats[0]=='v'):
            words=words[1:]
            nats=nats[1:]
            if(len(words)<1):
                continue
        
        out = ''.join(words)
        
        if(len(words)==1 and nats[0]=='vn'):
            none_flag = False
        if(len(nats)>=4 and ('vn' in set(nats) or 'v' in set(nats))):
            none_flag = False
        if(len(nats)>=1 and nats[0]=='v' and len(words[0])==1):
            none_flag = False
        if(len(nats)>=3 and nats[0] in noun_set and ('vn' in set(nats[1:-1]) or 'v' in set(nats[1:-1])) and nats[-1] in noun_set):##filter out nvn
            none_flag = False
        if(len(words)==1 and len(words[0])==1):
            none_flag = False
        if(none_flag and len(out)<7):
            out_list.append(out)
        
    
    return out_list

## on our news data

In [84]:
print_title_content_key_words(index=99)

title: 中城投资2016年上半年营收4.1亿元净赚1.2亿元 

content: 　　 9月26日消息，中城投资（证券代码：833880）近日公布的2016年上半年报告显示，报告期内实现营收4.08亿元，较上年同期增长8.35%；归属于挂牌公司股东的净利润为1.15亿元，较上年同期下降12.75%；基本每股收益为0.07元，较上年同期下降12.50%。 　　截止2016年6月30日，中城投资资产总计为67.94亿元，较本期期初下降19.48%；资产负债率为67.29%，较本期期初72.56%，下滑5.27个百分点。经营活动产生的现金流量净额本期为4.58亿元，上年同期为1.53亿元。 　　报告期内，中城投资经营情况变化的主要原因为市场处于利率下行通道，公司投资的收益率水平下降；中城投资降低了杠杆，同时也降低了自有资金投资规模，自有投资规模由年初的43.54亿元下降至30.69亿元，融资余额从年初的35.28亿元下降至23.06亿元；中城投资基金管理规模及收入较上年度同期有小幅增长，主要是由于实现的业绩报酬增加，但由于管理投入增加，业务成本较上年度增加，导致基金管理收入的毛利率下降。 　　中城投资经营性现金流分析：由于各板块业务收入增加，再加之提前收回财政返还，较上年度同期有大幅增加，2016年上半年，中城投资秉承经营安全的理念，保证充裕的现金流，同时积极回报股东，完成现金分配2.05亿元。 　　据挖贝新三板研究院资料显示，中城投资是一家专注于房地产领域的专业投资管理控股公司|中城投资以房地产全价值链和相关新产业链投资为特色|通过运营下辖多个业务板块，对不同的业务板块实施分类管理，为客户提供专业的咨询与管理服务，为投资人创造价值。 　　报告期内部分重要事项：  

key_words: 财政 全价值链 通道 现金分配 益率水平 价值
